### **0. Set-up**

In [1]:
# Import libraries and utils
%run '../../utils.ipynb'

In [2]:
# Get api key
load_dotenv()
OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY')

# Set client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
)

In [3]:
# Load dataframe
en_simlex = pd.read_csv("../../../data/dataset/cleaned-en-simlex-999.csv")

# Select subset
# en_simlex = en_simlex.head(150)

# Convert to tuple
tuples_list = list(zip(en_simlex['word1'], en_simlex['word2']))

In [4]:
# Show results
tuples_list

[('old', 'new'),
 ('smart', 'intelligent'),
 ('hard', 'difficult'),
 ('happy', 'cheerful'),
 ('hard', 'easy'),
 ('fast', 'rapid'),
 ('happy', 'glad'),
 ('short', 'long'),
 ('stupid', 'dumb'),
 ('weird', 'strange'),
 ('wide', 'narrow'),
 ('bad', 'awful'),
 ('easy', 'difficult'),
 ('bad', 'terrible'),
 ('hard', 'simple'),
 ('smart', 'dumb'),
 ('insane', 'crazy'),
 ('happy', 'mad'),
 ('large', 'huge'),
 ('hard', 'tough'),
 ('new', 'fresh'),
 ('sharp', 'dull'),
 ('quick', 'rapid'),
 ('dumb', 'foolish'),
 ('wonderful', 'terrific'),
 ('strange', 'odd'),
 ('happy', 'angry'),
 ('narrow', 'broad'),
 ('simple', 'easy'),
 ('old', 'fresh'),
 ('apparent', 'obvious'),
 ('inexpensive', 'cheap'),
 ('nice', 'generous'),
 ('weird', 'normal'),
 ('weird', 'odd'),
 ('bad', 'immoral'),
 ('sad', 'funny'),
 ('wonderful', 'great'),
 ('guilty', 'ashamed'),
 ('beautiful', 'wonderful'),
 ('confident', 'sure'),
 ('dumb', 'dense'),
 ('large', 'big'),
 ('nice', 'cruel'),
 ('impatient', 'anxious'),
 ('big', 'broad'),

### **1. Define and Evaluate Parameters**

In [5]:
# Define prompt
prompt = ("Rate the semantic similarity of each word pair on a scale from 0 to 5, "
          "where 0 represents no semantic similarity, and 5 represents perfect semantic similarity. "
          "Use two decimals. The response should strictly adhere to the structure: "
          "[('word1', 'word2', <score>), ('word3', 'word4', <score>), ...]. "
          "Do not provide additional explanations or context.")

In [6]:
# Define model
model = "meta-llama/llama-3.3-70b-instruct"

# Set sample size
sample_size = 15

# Delay between individual API calls
delay = 15.0

# Define number of sublists
n_sublists = 20

In [7]:
# Split list
chunks = split_into_n_lists(tuples_list, n_sublists)

# Count the number of lists
print(len(chunks))

20


In [8]:
# Print prompts for each chunk
print_prompts(chunks, prompt)

Rate the semantic similarity of each word pair on a scale from 0 to 5, where 0 represents no semantic similarity, and 5 represents perfect semantic similarity. Use two decimals. The response should strictly adhere to the structure: [('word1', 'word2', <score>), ('word3', 'word4', <score>), ...]. Do not provide additional explanations or context. --- ["('old', 'new'), ('smart', 'intelligent'), ('hard', 'difficult'), ('happy', 'cheerful'), ('hard', 'easy'), ('fast', 'rapid'), ('happy', 'glad'), ('short', 'long'), ('stupid', 'dumb'), ('weird', 'strange'), ('wide', 'narrow'), ('bad', 'awful'), ('easy', 'difficult'), ('bad', 'terrible'), ('hard', 'simple'), ('smart', 'dumb'), ('insane', 'crazy'), ('happy', 'mad'), ('large', 'huge'), ('hard', 'tough'), ('new', 'fresh'), ('sharp', 'dull'), ('quick', 'rapid'), ('dumb', 'foolish'), ('wonderful', 'terrific'), ('strange', 'odd'), ('happy', 'angry'), ('narrow', 'broad'), ('simple', 'easy'), ('old', 'fresh'), ('apparent', 'obvious'), ('inexpensive'

In [9]:
# Load encoding
encoding = tiktoken.get_encoding("cl100k_base")

# Count tokens per chunk
token_counts = count_tokens_with_tiktoken(chunks, prompt)

# Show results
print("Token counts for each formatted prompt:", token_counts)

Token counts for each formatted prompt: [439, 446, 427, 430, 421, 419, 434, 441, 440, 426, 426, 433, 436, 423, 429, 425, 416, 431, 437, 407]


### **2. Extract and Process Data**

In [10]:
# Get results from API
response = get_responses(chunks, prompt, model, sample_size, delay)

Processing: 100%|██████████| 300/300 [2:53:21<00:00, 34.67s/chunk]  

Total time taken: 10401.10 seconds


In [11]:
# Define filepath
file_path = '../../../data/llama-3.3-70b/response/en/f4.json'

# Check if file already exists
if not os.path.exists(file_path):
    with open(file_path, 'w') as f:
        json.dump(response, f)
        print("File saved successfully.")
else:
    print("File already exists. JSON was not saved to prevent overwriting.")

File saved successfully.


In [28]:
# Process data into dictionary
data_dict = process_responses(response)

# Check for values higher/lower then sample size
higher_lower_samples = {key: value for key, value in data_dict.items() if len(value) < sample_size or len(value) > sample_size}

# Show results
print(higher_lower_samples)

{('happy', 'glad'): [4.8, 4.7, 4.8, 4.8, 4.8, 4.8, 4.8, 4.6, 4.8, 4.8, 4.8, 4.7, 4.8, 4.8], ('happy', 'angry'): [0.2, 0.1, 0.1, 0.1, 0.1, 0.5, 0.2, 0.1, 0.1, 0.1, 0.2, 0.6, 0.1, 0.3], ('simple', 'easy'): [4.4, 4.4, 4.4, 4.4, 4.5, 4.4, 4.2, 4.5, 4.5, 4.5, 4.5, 4.2, 4.6, 4.4], ('apparent', 'obvious'): [4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.9, 4.8, 4.9, 4.9, 4.9, 4.9], ('nice', 'generous'): [4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 3.8, 4.2, 4.2, 4.2, 4.2, 4.1, 4.2, 4.2], ('confident', 'sure'): [4.7, 4.7, 4.5, 4.8, 4.6, 4.7, 4.4, 4.5, 4.5, 4.8, 4.6, 4.6, 4.8, 4.7], ('big', 'broad'): [4.3, 4.3, 4.1, 4.3, 4.3, 4.1, 3.9, 4.2, 4.3, 4.2, 4.3, 4.1, 4.3, 4.3], ('strong', 'proud'): [4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 3.7, 4.3, 4.2, 4.4, 4.2, 4.2, 4.2, 4.2], ('unnecessary', 'necessary'): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0], ('priest', 'monk'): [4.8, 4.8, 4.5, 4.8, 4.5, 4.5, 4.81, 4.5, 4.5, 4.81, 4.8, 4.5, 4.5, 4.5], ('anger', 'fury'): [4.9, 4.8, 4.9, 4.8, 4.9, 4.9, 4.92, 4.9

In [29]:
# Print duplicate word pairs
print_duplicate_word_pairs(en_simlex, data_dict)

Empty DataFrame
Columns: [Combined_Columns]
Index: []
Empty DataFrame
Columns: [Combined_Columns]
Index: []


In [30]:
# Convert dict to Pandas DataFrame
df = create_dataframe(data_dict)

# Show results
df

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
0,old,new,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,smart,intelligent,4.8,4.8,4.8,4.8,4.8,4.8,4.8,0.8,4.8,4.8,4.8,4.8,4.8,4.9,4.8
2,hard,difficult,4.9,4.9,4.7,4.6,4.9,4.6,4.6,0.6,4.9,4.9,4.9,4.9,4.6,4.8,4.6
3,happy,cheerful,4.7,4.8,4.9,4.8,4.7,4.8,4.8,4.0,4.7,4.7,4.7,4.7,4.8,4.7,4.8
4,hard,easy,0.1,0.1,0.1,0.2,0.1,0.2,0.2,0.2,0.1,0.1,0.1,0.1,0.2,0.1,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,join,acquire,3.5,3.0,4.0,4.4,4.4,3.0,4.0,4.0,3.0,4.2,4.2,3.5,4.2,4.4,NaN
1000,send,attend,2.5,2.0,3.2,3.6,3.1,2.5,3.0,3.0,2.0,3.4,3.4,3.5,2.5,3.0,3.8
1001,gather,attend,3.5,2.5,3.8,4.2,4.1,3.0,3.5,3.5,3.5,4.0,4.0,4.0,3.0,3.8,4.1
1002,absorb,withdraw,0.5,0.5,1.4,1.1,0.8,0.5,1.0,1.0,0.5,0.2,0.8,1.0,0.5,0.8,1.5


In [31]:
# Count null values
count_null_values = df.isnull().sum()

# Show results
print("Null value counts per column:", count_null_values)

Null value counts per column: word1                   0
word2                   0
similarity_score_1      0
similarity_score_2      5
similarity_score_3      5
similarity_score_4      5
similarity_score_5      5
similarity_score_6      5
similarity_score_7      5
similarity_score_8      5
similarity_score_9      5
similarity_score_10     5
similarity_score_11     5
similarity_score_12     5
similarity_score_13     5
similarity_score_14     5
similarity_score_15    75
dtype: int64


In [32]:
# Check for rows with at least one null value
rows_with_null = df[df.isnull().any(axis=1)]

# Show results
rows_with_null

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
6,happy,glad,4.8,4.7,4.8,4.8,4.8,4.8,4.8,4.6,4.8,4.8,4.8,4.7,4.8,4.8,NaN
26,happy,angry,0.2,0.1,0.1,0.1,0.1,0.5,0.2,0.1,0.1,0.1,0.2,0.6,0.1,0.3,NaN
28,simple,easy,4.4,4.4,4.4,4.4,4.5,4.4,4.2,4.5,4.5,4.5,4.5,4.2,4.6,4.4,NaN
30,apparent,obvious,4.9,4.9,4.9,4.9,4.9,4.9,4.9,4.9,4.9,4.8,4.9,4.9,4.9,4.9,NaN
32,nice,generous,4.2,4.2,4.2,4.2,4.2,4.2,3.8,4.2,4.2,4.2,4.2,4.1,4.2,4.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
966,absorb,learn,4.5,3.5,4.2,4.1,4.2,3.5,4.5,4.0,3.5,4.2,4.0,4.5,4.6,4.2,NaN
985,portray,decide,2.5,1.5,2.2,2.4,2.9,1.0,2.5,2.5,1.5,2.6,2.2,2.0,2.6,2.4,NaN
989,cooperate,join,4.0,3.5,4.2,4.1,4.5,3.0,4.2,4.0,4.0,4.2,4.0,4.0,4.2,4.5,NaN
990,arrange,require,3.0,2.0,3.2,3.5,3.8,2.0,3.5,3.0,2.5,3.0,3.2,2.5,3.4,3.5,NaN


In [33]:
# Extract missing word pairs
missing_word_pair_list = list(zip(rows_with_null['word1'], rows_with_null['word2']))

# Show results
missing_word_pair_list

[('happy', 'glad'),
 ('happy', 'angry'),
 ('simple', 'easy'),
 ('apparent', 'obvious'),
 ('nice', 'generous'),
 ('confident', 'sure'),
 ('big', 'broad'),
 ('strong', 'proud'),
 ('unnecessary', 'necessary'),
 ('priest', 'monk'),
 ('anger', 'fury'),
 ('sea', 'ocean'),
 ('death', 'burial'),
 ('music', 'melody'),
 ('beer', 'alcohol'),
 ('storm', 'hurricane'),
 ('flower', 'violet'),
 ('paper', 'cardboard'),
 ('motel', 'inn'),
 ('clothes', 'cloth'),
 ('book', 'bible'),
 ('alcohol', 'gin'),
 ('arm', 'wrist'),
 ('house', 'apartment'),
 ('glass', 'crystal'),
 ('wine', 'brandy'),
 ('creator', 'aker'),
 ('dinner', 'breakfast'),
 ('arm', 'uscle'),
 ('bubble', 'uds'),
 ('bread', 'flour'),
 ('gun', 'cannon'),
 ('church', 'choir'),
 ('door', 'cellar'),
 ('metal', 'aluminum'),
 ('cloud', 'fog'),
 ('boy', 'on'),
 ('attorney', 'lawyer'),
 ('clothes', 'fabric'),
 ('alcohol', 'cocktail'),
 ('molecule', 'atom'),
 ('lens', 'camera'),
 ('meal', 'dinner'),
 ('eye', 'tear'),
 ('rat', 'ouse'),
 ('creator', 'mak

In [ ]:
# Define file_path
file_path = '../../../data/prompt/en/llama-3.3-70b/f4.csv'

# Check if file already exists
if not os.path.exists(file_path):
    df.to_csv(file_path, index=False)
    print("File saved successfully.")
else:
    print("File already exists. Dataframe was not saved to prevent overwriting.")